In [14]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2

from histomicstk.preprocessing.color_normalization import deconvolution_based_normalization
from histomicstk.saliency.tissue_detection import (
    get_slide_thumbnail, get_tissue_mask)
from skimage.transform import resize
from skimage.exposure import rescale_intensity
from stardist.models import StarDist2D
from stardist.plot import render_label

from utils.helpers import compare_two_images, cut_out_image
from tiatoolbox.tools import stainnorm
from typing import List, Tuple

from utils.helpers import compare_two_images, cut_out_image

In [2]:
image_paths = sorted(["data/images/"+path for path in os.listdir("data/images") if not path.startswith(".")])
images = list(map(tifffile.imread, image_paths))

normalized_image_paths = sorted(["data/normalized_images/"+path for path in os.listdir("data/normalized_images/") if not path.startswith(".")])
normalized_images = list(map(tifffile.imread, normalized_image_paths))

mask_paths = sorted(["data/masks/"+path for path in os.listdir("data/masks") if not path.startswith(".")])
masks = list(map(tifffile.imread, mask_paths))
data = list(zip(images, masks))

In [12]:
test_image, test_mask = data[0]


In [4]:
model = StarDist2D.from_pretrained('2D_versatile_he')

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


In [13]:
test_image_norm = rescale_intensity(test_image, out_range=(0, 1))
test_labels, test_dict = model.predict_instances(test_image_norm, axes='YXC', prob_thresh=0.05, nms_thresh=0.3, return_labels=True)
test_labels = cut_out_image(test_labels, test_mask)

In [15]:
nuclei_mask = np.uint8(test_labels)

In [22]:
# # Create distance map
# distance_map = cv2.distanceTransform(nuclei_mask, cv2.DIST_L2, 5)
# surface = -distance_map
# compare_two_images(surface, distance_map)

In [23]:
# # Define markers
# ret, markers = cv2.connectedComponents(nuclei_mask)
# markers = markers.astype(np.int32)
# plt.imshow(markers)
# plt.axis(False)
# plt.show()

In [ ]:
dummy_img = np.zeros((*nuclei_mask.shape, 3), dtype=np.uint8)
labeled_test_img = cv2.watershed(dummy_img, markers)

In [24]:
# compare_two_images(test_image, labeled_test_img)